**Goal:** To train the ASR model and validate on the LJSPEECH dataset using WER metric.






**Approach 1:**
Train WAV2VEC2 model on a custom dataset- LJSPEECH

In [44]:
%%capture
!pip install datasets
!pip install transformers
!pip install huggingface_hub
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install glob2

Splitting the dataset into train(90%) and test(10%).

In [2]:
import datasets
from datasets import load_dataset, load_metric, Audio
train_ds = datasets.load_dataset('lj_speech', split='train[:90%]')
test_ds = datasets.load_dataset('lj_speech', split='train[90%:]')

Reusing dataset lj_speech (/root/.cache/huggingface/datasets/lj_speech/main/1.1.0/f4518d8fe24e62a9045ac697d23037d073cc76202777ee14267664978c222c2f)


In [48]:
test_ds

Dataset({
    features: ['audio', 'file', 'normalized_text'],
    num_rows: 1310
})

In [ ]:
test_ds = test_ds.remove_columns(["text", "id"])
train_ds = train_ds.remove_columns(["text", "id"])
train_ds, test_ds

Removing special characters from attached transcript(reference text).

In [12]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\)\(\'\“\[\]\'à\â\è\é\ê\ü\’\”]'

def remove_special_characters(batch):
    batch["normalized_text"] = re.sub(chars_to_ignore_regex, '', batch["normalized_text"]).lower()
    return batch

train_ds = train_ds.map(remove_special_characters)
test_ds = test_ds.map(remove_special_characters)

  0%|          | 0/11790 [00:00<?, ?ex/s]

  0%|          | 0/1310 [00:00<?, ?ex/s]

In [13]:
def extract_all_chars(batch):
  all_text = " ".join(batch["normalized_text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs_tr = train_ds.map(extract_all_chars, batched=True, batch_size=-1, 
                         keep_in_memory=True, remove_columns=train_ds.column_names)

vocabs_test = test_ds.map(extract_all_chars, batched=True, batch_size=-1, 
                          keep_in_memory=True, remove_columns=test_ds.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Building the vocab.

In [14]:
vocab_list = list(set(vocabs_tr["vocab"][0]) | set(vocabs_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 12,
 'a': 6,
 'b': 25,
 'c': 0,
 'd': 20,
 'e': 26,
 'f': 17,
 'g': 16,
 'h': 7,
 'i': 1,
 'j': 23,
 'k': 4,
 'l': 11,
 'm': 5,
 'n': 22,
 'o': 18,
 'p': 21,
 'q': 13,
 'r': 19,
 's': 8,
 't': 15,
 'u': 9,
 'v': 14,
 'w': 24,
 'x': 2,
 'y': 10,
 'z': 3}

Adding unknown and padding tokens.

In [15]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

29

In [16]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [17]:
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, 
                                             padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [31]:
train_ds[0]["audio"]

{'array': array([-7.3242188e-04, -7.6293945e-04, -6.4086914e-04, ...,
         7.3242188e-04,  2.1362305e-04,  6.1035156e-05], dtype=float32),
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ001-0001.wav',
 'sampling_rate': 22050}

Audio resampling to 16Khz.

In [20]:
train_ds = train_ds.cast_column("audio", Audio(sampling_rate=16_000))
test_ds = test_ds.cast_column("audio", Audio(sampling_rate=16_000))

In [21]:
train_ds[0]["audio"]

{'array': array([-0.00064146, -0.00074657, -0.00068768, ...,  0.00068341,
         0.00014045,  0.        ], dtype=float32),
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ001-0001.wav',
 'sampling_rate': 16000}

In [22]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train_ds)-1)

print(train_ds[rand_int]["normalized_text"])
ipd.Audio(data=train_ds[rand_int]["audio"]["array"], autoplay=True, rate=16000)

and may thus be easily transported to all parts of the plant


In [23]:
rand_int = random.randint(0, len(train_ds)-1)

print("Target text:", train_ds[rand_int]["normalized_text"])
print("Input array shape:", train_ds[rand_int]["audio"]["array"].shape)
print("Sampling rate:", train_ds[rand_int]["audio"]["sampling_rate"])

Target text: it is not known whether the application for post office box two nine one five listed a hidell as a person entitled to receive mail at this box
Input array shape: (154481,)
Sampling rate: 16000


In [24]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["normalized_text"]).input_ids
    return batch

In [25]:
train_ds = train_ds.map(prepare_dataset, remove_columns=train_ds.column_names)
test_ds = test_ds.map(prepare_dataset, remove_columns=test_ds.column_names)

  0%|          | 0/11790 [00:00<?, ?ex/s]

  0%|          | 0/1310 [00:00<?, ?ex/s]

In [26]:
max_input_length_in_sec = 5.0
train_ds = train_ds.filter(
    lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, 
    input_columns=["input_length"])

  0%|          | 0/12 [00:00<?, ?ba/s]

Data collator. 

[Reference code](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81)



In [27]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [28]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Word error rate.

In [29]:
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

In [30]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

The pretrained WAV2VEC2-XLSR model.

In [31]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Downloading:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_q.weight', 'project_hid.bias', 'project_hid.weight', 'quantizer.weight_proj.weight', 'quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [32]:
model.freeze_feature_extractor()

set-up the training arguments.

In [36]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir='/',
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  #fp16=True,
  save_steps=40,
  eval_steps=40,
  logging_steps=40,
  learning_rate=3e-4,
  warmup_steps=50,
  save_total_limit=2,
  #push_to_hub=True,
)

In the next step, the model will be trained on the training dataset and compute the WER on the validation dataset.

In [37]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=processor.feature_extractor,
)

Due to memory limitations, I have not completed the training, as it requires 351hours to train (without GPU). I have tried use the GPU, but the colab memory is getting full. Probably, we should use colab pro.

In [38]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running training *****
  Num examples = 3000
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 2820
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:984: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: Us

Step,Training Loss,Validation Loss


KeyboardInterrupt: ignored

**Approach 2:** Validating pre-trained and fine-tuned model.

Here, I have used wav2vec2 pretrained and fine-tuned model on 960 hours of Librispeech on 16kHz sampled speech audio. And, tested it on the LJSPEECH(10%) dataset. The WER was 1.019 (see the bottom of the output cell). 

In [11]:
# Importing libraries
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import soundfile as sf
import torch
from jiwer import wer

# Declaring validation dataset.
test_ds = datasets.load_dataset('lj_speech', split='train[90%:]')

# Resampling to 16KHz to match with our model input requirements.
test_ds = test_ds.cast_column("audio", Audio(sampling_rate=16_000)) 

# The base model pretrained and fine-tuned on 960 hours of Librispeech on 16kHz sampled speech audio.
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

# This function adds a 'speech' array attribute to the dataset.
def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch

test_ds = test_ds.map(map_to_array)

def map_to_pred(batch):
    input_values = processor(batch["speech"], return_tensors="pt", padding="longest").input_values
    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    batch["transcription"] = transcription
    return batch

result = test_ds.map(map_to_pred, batched=True, batch_size=1, remove_columns=["speech"])

# computing Word Error Rate.
print("WER:", wer(result["normalized_text"], result["transcription"]))

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1310 [00:00<?, ?ex/s]

  0%|          | 0/1310 [00:00<?, ?ba/s]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_ra

WER: 1.0194116837226377
